# MWS Secondary Target Selection

## MWS_SPECIAL_GC_SV

Maintainers:
  - Ting Li (selection/catalog)
  - Andrew Cooper (secondary target code)

Membership probability is calculated by Vasiliev 2018 for 150 GC (based on the spatial distribution and proper motion) in https://arxiv.org/abs/1807.09775

In this catalog we merge all the stars with membership probability > 0.5 into one list (0.5 is an arbitrary pick and we can easily change to a higher or lower threshold).

In [11]:
!pwd
!ls ../raw

/global/projecta/projectdirs/desi/external/mws/sv/MWS_SPECIAL_GC_SV/1/code
gc_p0_5_gaia_4DESI_v0.fits


In [1]:
# Parameters

target_class   = 'MWS_SPECIAL_GC_SV'
input_filename = 'gc_p0_5_gaia_4DESI_v0.fits'

REF_CAT        = '?'
REF_EPOCH      = 2015.5
BRIGHT_LIMIT   = 16.0
FAINT_LIMIT    = 20.0

SELECT_MAG     = 'phot_g_mean_mag'

colmap = {
    'ra':        'RA',
    'dec':       'DEC',
    'source_id': 'REF_ID'
}

In [2]:
from   importlib import reload
import sys
PYPATH = '/project/projectdirs/desi/external/mws/py/'
if PYPATH not in sys.path: sys.path.append(PYPATH)
import mwsecondary.mwsecondary as mws
reload(mws)

import os
import sys
import numpy as np

targets = mws.SecondaryTargets(target_class,input_filename,ref_cat=REF_CAT,ref_epoch=REF_EPOCH)
targets.print_info()

mask = np.repeat(True,len(targets.data))
mask &= targets.select_finite_mag_range(SELECT_MAG,faint_limit=FAINT_LIMIT,bright_limit=BRIGHT_LIMIT)
mask &= targets.select_finite_range('dec',lower_limit=-30.0,verbose=True)
mask &= targets.select_unique('source_id',verbose=True)

# Adjust to secondary target data model and write output
output = targets.get_output(mask,cols=colmap,write=True)
print(output[0:3])

Reading input: /global/projecta/projectdirs/desi/external/mws/sv/MWS_SPECIAL_GC_SV/1/raw/gc_p0_5_gaia_4DESI_v0.fits
Read 183758 rows
First 3 rows:
    source_id             ra              dec        ...   ebv   memberprob
------------------ ---------------- ---------------- ... ------- ----------
159501173974695040 71.5000241420384 31.3407135079063 ... 1.16956   0.759413
159501547636737024 71.5102834721046 31.3441511214088 ... 1.17215  0.8645645
159501581996498688 71.5242665415407 31.3625396867982 ... 1.18219  0.9918003

Column names:
['source_id', 'ra', 'dec', 'pmra', 'pmdec', 'phot_g_mean_mag', 'phot_bp_mean_mag', 'phot_rp_mean_mag', 'ebv', 'memberprob']


Mag range:  9.34 < phot_g_mean_mag < 21.00
Mags not finite: 0
Selecting 146123 targets from 183758 (79.52%)
Selecting 70583 from 183758 (38.41%) on dec
183758 of 183758 targets have unique source_id
Removing 0 duplicates

Writing 53328 rows of output to /global/projecta/projectdirs/desi/external/mws/sv/MWS_SPECIAL_GC_SV/1/indata/M